In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
df=pd.read_excel('Acoustic_Extinguisher_Fire_Dataset.xlsx')

In [12]:
df

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS
0,1,gasoline,10,96,0.0,75,0
1,1,gasoline,10,96,0.0,72,1
2,1,gasoline,10,96,2.6,70,1
3,1,gasoline,10,96,3.2,68,1
4,1,gasoline,10,109,4.5,67,1
...,...,...,...,...,...,...,...
17437,7,lpg,190,86,2.2,5,0
17438,7,lpg,190,84,2.0,4,0
17439,7,lpg,190,80,1.5,3,0
17440,7,lpg,190,76,0.4,2,0


In [13]:
df.columns

Index(['SIZE', 'FUEL', 'DISTANCE', 'DESIBEL', 'AIRFLOW', 'FREQUENCY',
       'STATUS'],
      dtype='object')

In [14]:
x=df.drop('STATUS',axis=1)
y=df['STATUS']

In [17]:
x['FUEL'].value_counts()

gasoline    5130
thinner     5130
kerosene    5130
lpg         2052
Name: FUEL, dtype: int64

In [20]:
x['SIZE'].value_counts()

1    3078
2    3078
3    3078
4    3078
5    3078
6    1026
7    1026
Name: SIZE, dtype: int64

In [22]:
y.value_counts()

0    8759
1    8683
Name: STATUS, dtype: int64

In [24]:
x.isnull().sum()

SIZE         0
FUEL         0
DISTANCE     0
DESIBEL      0
AIRFLOW      0
FREQUENCY    0
dtype: int64

In [26]:
y.isnull().sum()

0

In [29]:
# Lebal Encoding
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for col in x.columns:
    x[col]=le.fit_transform(x[col])

In [30]:
x.describe()

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY
count,17442.000000,17442.000000,17442.000000,17442.000000,17442.000000,17442.000000
mean,2.411765,1.411765,9.000000,21.431774,51.943470,26.500000
std,1.750977,1.191120,5.477383,8.036195,37.038853,15.586231
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,4.000000,15.000000,22.000000,13.000000
50%,2.000000,1.000000,9.000000,20.000000,43.000000,26.500000
75%,4.000000,3.000000,14.000000,29.000000,85.000000,40.000000
max,6.000000,3.000000,18.000000,38.000000,126.000000,53.000000


In [31]:
#Data cleaning
from sklearn.ensemble import IsolationForest
iso=IsolationForest(random_state=0,contamination=0.05)
clean=iso.fit_predict(x,y)

C:\ProgramData\anacondaa\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [32]:
x=x[clean==1]
y=y[clean==1]

In [33]:
x.shape

(16569, 6)

In [34]:
y.shape

(16569,)

In [160]:
#scaling
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
x_scaled=scaler.fit_transform(x)

In [161]:
#Checking Accuracy

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,random_state=0)

In [162]:
x_train.shape

(12426, 6)

In [163]:
x_test.shape

(4143, 6)

In [164]:
from sklearn.metrics import accuracy_score

In [165]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


In [166]:
clf_log=LogisticRegression(random_state=0)
clf_dt=DecisionTreeClassifier(random_state=0)
clf_knn=KNeighborsClassifier(n_neighbors=5)
clf_nb=GaussianNB()
clf_svm=SVC(kernel='rbf',random_state=0)
clf_rf=RandomForestClassifier(random_state=0)

In [167]:
clf_log.fit(x_train,y_train)
clf_dt.fit(x_train,y_train)
clf_knn.fit(x_train,y_train)
clf_nb.fit(x_train,y_train)
clf_svm.fit(x_train,y_train)
clf_rf.fit(x_train,y_train);

In [168]:
y_pred_log=clf_log.predict(x_test)
y_pred_dt=clf_dt.predict(x_test)
y_pred_knn=clf_knn.predict(x_test)
y_pred_nb=clf_nb.predict(x_test)
y_pred_svm=clf_svm.predict(x_test)
y_pred_rf=clf_rf.predict(x_test)

In [169]:
print('LogR',accuracy_score(y_test,y_pred_log))
print('Dt',accuracy_score(y_test,y_pred_dt))
print('KNN:',accuracy_score(y_test,y_pred_knn))
print('NB',accuracy_score(y_test,y_pred_nb))
print('SVM',accuracy_score(y_test,y_pred_svm))
print('RFR',accuracy_score(y_test,y_pred_rf))

LogR 0.872073376780111
Dt 0.9669321747525947
KNN: 0.9623461259956553
NB 0.8672459570359643
SVM 0.9379676562877142
RFR 0.9577600772387159


In [170]:
# Hyperparameter
from sklearn.model_selection import GridSearchCV
para={
    'random_state':[0,1,2,3,4,5],
    'n_estimators':[100,200,300,400,500]
}

In [171]:
grid= GridSearchCV(clf_rf,param_grid=para,cv=6,
                   scoring='accuracy')

In [172]:
grid.fit(x_train,y_train)

GridSearchCV(cv=6, estimator=RandomForestClassifier(random_state=0),
             param_grid={'n_estimators': [100, 200, 300, 400, 500],
                         'random_state': [0, 1, 2, 3, 4, 5]},
             scoring='accuracy')

In [173]:
y_pred_grid=grid.predict(x_test)
accuracy_score(y_test,y_pred_grid)

0.9589669321747526